<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/master/notebooks/build_index_and_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id 10LV9QbZOkUyOzR4nh8hxesoKJhpmvpM9 # citation vectors
!gdown --id 114nknnGwnY1nymCG1aPAhso7FlarZ8UP # saved_model files
!unzip tfworld.zip

Downloading...
From: https://drive.google.com/uc?id=10LV9QbZOkUyOzR4nh8hxesoKJhpmvpM9
To: /content/CitationSimilarityVectors106Epochs.npy
2.59GB [00:23, 110MB/s] 
Downloading...
From: https://drive.google.com/uc?id=114nknnGwnY1nymCG1aPAhso7FlarZ8UP
To: /content/tfworld.zip
411MB [00:13, 31.1MB/s]
Archive:  tfworld.zip
   creating: gdrive/My Drive/tfworld/
   creating: gdrive/My Drive/tfworld/inference_model/
   creating: gdrive/My Drive/tfworld/inference_model/variables/
  inflating: gdrive/My Drive/tfworld/inference_model/variables/variables.data-00000-of-00002  
  inflating: gdrive/My Drive/tfworld/inference_model/variables/variables.data-00001-of-00002  
  inflating: gdrive/My Drive/tfworld/inference_model/variables/variables.index  
   creating: gdrive/My Drive/tfworld/inference_model/assets/
  inflating: gdrive/My Drive/tfworld/inference_model/saved_model.pb  
   creating: gdrive/My Drive/tfworld/citations_projection_model/
   creating: gdrive/My Drive/tfworld/citations_projection

In [3]:
%tensorflow_version 2.x
import numpy as np
import tensorflow as tf
from time import time
from tqdm import tqdm_notebook as tqdm
print('TensorFlow:', tf.__version__)

TensorFlow 2.x selected.
TensorFlow: 2.1.0-rc1


In [4]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  ['10.82.166.58:8470']
INFO:tensorflow:Initializing the TPU system: 10.82.166.58:8470


INFO:tensorflow:Initializing the TPU system: 10.82.166.58:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [5]:
workers = ['/job:worker/replica:0/task:0/device:TPU:'+str(i) for i in range(8)]
workers

['/job:worker/replica:0/task:0/device:TPU:0',
 '/job:worker/replica:0/task:0/device:TPU:1',
 '/job:worker/replica:0/task:0/device:TPU:2',
 '/job:worker/replica:0/task:0/device:TPU:3',
 '/job:worker/replica:0/task:0/device:TPU:4',
 '/job:worker/replica:0/task:0/device:TPU:5',
 '/job:worker/replica:0/task:0/device:TPU:6',
 '/job:worker/replica:0/task:0/device:TPU:7']

In [0]:
class Index:
    def __init__(self, embeddings, worker, normalize_embeddings=True):
        self.embeddings = embeddings
        if normalize_embeddings:
           self.embeddings = tf.math.l2_normalize(self.embeddings, axis=1)
        self.worker = worker

    @tf.function
    def search(self, query_vector, top_k=None, ):
      with tf.device(worker):
        dot_product = tf.reduce_sum(tf.multiply(self.embeddings, query_vector), axis=1, keepdims=True)
        distances = -1*dot_product
        sorted_indices =  tf.argsort(distances)
        if top_k:
            sorted_indices = sorted_indices[..., :top_k]
        nearest_distances = tf.reshape(tf.gather(distances[0], sorted_indices), shape=[-1, 1])
        return nearest_distances[..., 0], sorted_indices[0]

In [7]:
citations_embeddings = np.load('CitationSimilarityVectors106Epochs.npy')
citations_embeddings.shape

(1262996, 512)

In [15]:
# Discarding last 4 vectors to make number of vectors divisible by 8
citations_embeddings = np.split(citations_embeddings[:-4], 8, axis=0)
vecs_per_index = citations_embeddings[0].shape[0]
print('Vectors per index :', vecs_per_index)

Vectors per index : 157874


In [28]:
## Place 1/8 of total embeddings on each TPU core
indices = []
for i, worker in enumerate(workers):
  with tf.device(worker):
    print('Building index with {} vectors on {}'.format(citations_embeddings[i].shape[0],worker))
    indices.append(Index(citations_embeddings[i], worker, normalize_embeddings=True))

Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:0
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:1
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:2
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:3
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:4
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:5
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:6
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:7


In [0]:
def search(xq):
  D, I = [], []
  for i in range(8):
    print('Search running in index: {}'.format(indices[i].worker))
    d, idx = indices[i].search(xq, 1)
    D.append(d.numpy()[0])
    I.append(i*vecs_per_index + idx.numpy()[0])

  id_sorted = np.argsort(D)
  D = np.array(D)[id_sorted]
  I = np.array(I)[id_sorted]
  return D, I

In [31]:
n = 1
split = 0 # [0, 7] . # Pick nth vector from given split
actual_n = vecs_per_index*split + n

xq = tf.nn.l2_normalize(citations_embeddings[split][n])
xq = tf.reshape(xq, [1, -1])

s = time()
D, I = search(xq)
e = time()

print('\nActual ID   :', actual_n)
print('Result ID   :', I[0], '\n')

print('Neighbours   :', I )
print('Distances   :', np.round(D, 4))
print('\nTime taken  :', np.round(e-s, 2), 'secs')
# First search runs slow, because tf.function traces the function
# only for the first invocation, following invocations should run fine

Search running in index: /job:worker/replica:0/task:0/device:TPU:0
Search running in index: /job:worker/replica:0/task:0/device:TPU:1
Search running in index: /job:worker/replica:0/task:0/device:TPU:2
Search running in index: /job:worker/replica:0/task:0/device:TPU:3
Search running in index: /job:worker/replica:0/task:0/device:TPU:4
Search running in index: /job:worker/replica:0/task:0/device:TPU:5
Search running in index: /job:worker/replica:0/task:0/device:TPU:6
Search running in index: /job:worker/replica:0/task:0/device:TPU:7

Actual ID   : 1
Result ID   : 157874 

Neighbours   : [ 157874 1105118  947244  315748  473622  631496       0  789370]
Distances   : [-0.2013 -0.0311 -0.023   0.009   0.0694  0.0781  0.1051  0.2201]

Time taken  : 0.18 secs


In [39]:
citations_embeddings[0][1] - citations_embeddings[0][1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [34]:
tf.norm(indices[0].embeddings[1])

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [0]:
Actual ID   : 1115204
Result ID   : 1115204 

Neighbours   : [1115204  458063 1076920  776066  888519  161765  109981  579263]
Distances   : [0.     0.4552 0.5766 0.8795 0.9011 1.3016 1.3716 1.3912]

Time taken  : 0.22 secs

#### Checking accuracy

In [0]:
n_test = 5000
random_n = np.random.randint(0, vecs_per_index, n_test)
random_split = np.random.randint(0, 8, n_test)

In [0]:
y_true= []
y_pred = []
s = time()
for n, split in tqdm(zip(random_n, random_split), total=n_test):
  xq = tf.nn.l2_normalize(u_embeddings[split][n]) + tf.nn.l2_normalize(v_embeddings[split][n])
  xq = tf.reshape(xq, [1, -1])
  actual_n = vecs_per_index*split + n
  D, I = [], []
  for i in range(8):
    d, idx = indices[i].search(xq, 1)
    D.append(d.numpy()[0])
    I.append(i*vecs_per_index + idx.numpy()[0])
  id_sorted = np.argsort(D)
  y_pred.append(np.array(I)[id_sorted])
  y_true.append(actual_n)
e = time()
y_true = np.array(y_true)
y_pred = np.array(y_pred)
print('\nTime taken per search    :', np.round(e-s, 2) / n_test, 'secs')
print('Accuracy                 :', np.sum(y_true == y_pred[:, 0]) / n_test)



Time taken per search    : 0.24776199999999998 secs
Accuracy                 : 1.0
